# ERNIE-Bot Chat

[ERNIE-Bot](https://cloud.baidu.com/doc/WENXINWORKSHOP/s/jlil56u11) is a large language model developed by Baidu, covering a huge amount of Chinese data.
This notebook covers how to get started with ErnieBot chat models.

In [2]:
from langchain.chat_models import ErnieBotChat
from langchain.schema import HumanMessage

In [ ]:
chat = ErnieBotChat(ernie_client_id='your_ak', ernie_client_secret='your_sk')

or you can set `client_id` and `client_secret` in your environment variables
```bash
export ERNIE_CLIENT_ID=YOUR_CLIENT_ID
export ERNIE_CLIENT_SECRET=YOUR_CLIENT_SECRET
```

In [7]:
chat([
    HumanMessage(content='hello there, who are you?')
])

AIMessage(content='My name is dst. As an artificial intelligence language model, I have no real form and identity, and my purpose is to help users get information and answers. What can I do for you?', additional_kwargs={}, example=False)

In [10]:
chat.generate(messages=[[
    HumanMessage(content='hello there, who are you?')
]])

LLMResult(generations=[[ChatGeneration(text='Hello, I am an artificial intelligence language model. My purpose is to help users obtain information and answer their questions. What can I do for you?', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='Hello, I am an artificial intelligence language model. My purpose is to help users obtain information and answer their questions. What can I do for you?', additional_kwargs={}, example=False))]], llm_output={}, run=[RunInfo(run_id=UUID('3e0350a6-daa6-4754-9d27-a014ccf7bd0a'))])

In [8]:
# aio functions
import asyncio

res = chat.stream([HumanMessage(content="hi")], stream=True)
for r in res:
    print("chat resp1:", r)


async def run_aio_generate():
    resp = await chat.agenerate(messages=[[HumanMessage(content="write a 20 words sentence about sea.")]])
    print(resp)
        
await run_aio_generate()

async def run_aio_stream():
    async for res in chat.astream([HumanMessage(content="write a 20 words sentence about sea.")]):
        print("astream", res)
        
await run_aio_stream()

chat resp1: content='您好，您似乎输入' additional_kwargs={} example=False
chat resp1: content='了一个话题标签，需要我帮忙解答吗？' additional_kwargs={} example=False
chat resp1: content='请提供更多的信息给我。' additional_kwargs={} example=False
chat resp1: content='' additional_kwargs={} example=False
generations=[[ChatGeneration(text='The sea is a vast expanse of water, stretching as far as the eye can see. It is a source of inspiration and adventure for many people, and a place where they can relax and unwind.', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='The sea is a vast expanse of water, stretching as far as the eye can see. It is a source of inspiration and adventure for many people, and a place where they can relax and unwind.', additional_kwargs={}, example=False))]] llm_output={} run=[RunInfo(run_id=UUID('d73cca9d-bf7d-4bb9-b80c-2421d801be74'))]
astream content='大海是地球上最' additional_kwargs={} example=False
astream content='广阔的水域之一，它拥有美丽的海岸线和各种奇妙的海洋生物。' additional_kwargs={} example=False

## Function Call
- use functions call with LLM by chatModels

In [ ]:
_FUNCTIONS: Any = [
    {
        "name": "format_person_info",
        "description": (
            "Output formatter. Should always be used to format your response to the"
            " user."
        ),
        "parameters": {
            "title": "Person",
            "description": "Identifying information about a person.",
            "type": "object",
            "properties": {
                "name": {
                    "title": "Name",
                    "description": "The person's name",
                    "type": "string",
                },
                "age": {
                    "title": "Age",
                    "description": "The person's age",
                    "type": "integer",
                },
                "fav_food": {
                    "title": "Fav Food",
                    "description": "The person's favorite food",
                    "type": "string",
                },
            },
            "required": ["name", "age"],
        },
    },
    {
        "name": "get_current_temperature",
        "description": ("Used to get the location's temperature."),
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "city name",
                },
                "unit": {
                    "type": "string",
                    "enum": ["centigrade", "Fahrenheit"],
                },
            },
            "required": ["location", "unit"],
        },
        "responses": {
            "type": "object",
            "properties": {
                "temperature": {
                    "type": "integer",
                    "description": "city temperature",
                },
                "unit": {
                    "type": "string",
                    "enum": ["centigrade", "Fahrenheit"],
                },
            },
        },
    },
]


In [ ]:
chat = ErnieBotChat()

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, FunctionMessage, HumanMessage, SystemMessage
from langchain.chains.openai_functions import create_openai_fn_chain

prompt = ChatPromptTemplate(
    messages=[
        HumanMessage(content="What's the temperature in Shanghai today?"),
        AIMessage(
            content="",
            additional_kwargs={
                "function_call": {
                    "name": "get_current_temperature",
                    "thoughts": "i will use get_current_temperature "
                    "to resolve the questions",
                    "arguments": '{"location":"Shanghai","unit":"centigrade"}',
                }
            },
        ),
        FunctionMessage(
            name="get_current_weather",
            content='{"temperature": "25", \
                            "unit": "摄氏度", "description": "晴朗"}',
        ),
    ]
)
llm_chain = create_openai_fn_chain(
    _FUNCTIONS,
    chat,
    prompt,
    output_parser=None,
)
resp = llm_chain.generate([{}])
print("resp", resp)